# Módulo 5 Tarefa 1
## Base de nascidos vivos do DataSUS
O DataSUS disponibiliza diversos arquivos de dados com relação a seus segurados, conforme a [lei da transparência de informações públicas](https://www.sisgov.com/transparencia-acesso-informacao/#:~:text=A%20Lei%20da%20Transpar%C3%AAncia%20(LC,em%20um%20site%20na%20internet.). 

Essas informações podem ser obtidas pela internet [aqui](http://www2.datasus.gov.br/DATASUS/index.php?area=0901&item=1). Como o processo de obtenção desses arquivos foge um pouco do nosso escopo, deixamos o arquivo ```SINASC_RO_2019.csv``` já como vai ser encontrado no DataSUS. O dicionário de dados está no arquivo ```estrutura_sinasc_para_CD.pdf``` (o nome do arquivo tal qual no portal do DataSUS).

### Nosso objetivo
Queremos deixar uma base organizada para podermos estudar a relação entre partos com risco para o bebê e algumas condições como tempo de parto, consultas de pré-natal etc.

#### Preparação da base
1. Carregue a base 'SINASC_RO_2019.csv'. Conte o número de registros e o número de registros não duplicados da base. Dica: você aprendeu um método que remove duplicados, encadeie este método com um outro método que conta o número de linhas. **Há linhas duplicadas?**  

2. Conte o número de valores *missing* por variável.  

3. Ok, no item anterior você deve ter achado pouco prático ler a informação de tantas variáveis, muitas delas nem devem ser interesantes. Então crie uma seleção dessa base somente com as colunas que interessam. São elas:
``` 
['LOCNASC', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO', 
    'GESTACAO', 'GRAVIDEZ', 'CONSULTAS', 'APGAR5'] 
```
Refaça a contagem de valores *missings*.  

4. Apgar é uma *nota* que o pediatra dá ao bebê quando nasce de acordo com algumas características associadas principalmente à respiração. Apgar 1 e Apgar 5 são as notas 1 e 5 minutos do nascimento. Apgar5 será a nossa variável de interesse principal. Então remova todos os registros com Apgar5 não preenchido. Para esta seleção, conte novamente o número de linhas e o número de *missings*.  

5. observe que as variáveis ```['ESTCIVMAE', 'CONSULTAS']``` possuem o código ```9```, que significa *ignorado*. Vamos assumir que o não preenchido é o mesmo que o código ```9```.<br>
6. Substitua os valores faltantes da quantitativa (```QTDFILVIVO```) por zero.  
7. Das restantes, decida que valore te parece mais adequado (um 'não preenchido' ou um valor 'mais provável' como no item anterior) e preencha. Justifique. Lembre-se de que tratamento de dados é trabalho do cientista, e que estamos tomando decisões a todo o momento - não há necessariamente certo e errado aqui.  
8. O Apgar possui uma classificação indicando se o bebê passou por asfixia:
- Entre 8 e 10 está em uma faixa 'normal'. 
- Entre 6 e 7, significa que o recém-nascido passou por 'asfixia leve'. 
- Entre 4 e 5 significa 'asfixia moderada'.
- Entre 0 e 3 significa 'asfixia severa'.  

Crie uma categorização dessa variável com essa codificação e calcule as frequências dessa categorização.  
<br>
9. Renomeie as variáveis para que fiquem no *snake case*, ou seja, em letras minúsculas, com um *underscore* entre as palávras. Dica: repare que se você não quiser criar um *dataframe* novo, você vai precisar usar a opção ```inplace = True```.

In [2]:
import pandas as pd
import requests

# 1) seu código aqui
path = '/content/sample_data/SINASC_RO_2019.csv'
sinasc = pd.read_csv(path)
print(sinasc.shape)
sinasc.drop_duplicates().shape
# Não há duplicados

(27028, 69)


(27028, 69)

In [3]:
# 2) seu código aqui
sinasc.isna().sum()


ORIGEM          0
CODESTAB      115
CODMUNNASC      0
LOCNASC         0
IDADEMAE        0
             ... 
munResUf        0
munResLat       1
munResLon       1
munResAlt       1
munResArea      1
Length: 69, dtype: int64

In [4]:
# 3) seu código aqui
df = sinasc[["LOCNASC", "IDADEMAE", "ESTCIVMAE", "ESCMAE", "QTDFILVIVO", "GESTACAO", "GRAVIDEZ", "CONSULTAS", "APGAR5"]]
df.isna().sum()

LOCNASC          0
IDADEMAE         0
ESTCIVMAE      317
ESCMAE         312
QTDFILVIVO    1573
GESTACAO      1232
GRAVIDEZ        79
CONSULTAS        0
APGAR5         103
dtype: int64

In [5]:
# 4) seu código aqui
df = df.dropna(subset=["APGAR5"])
df.isna().sum()


LOCNASC          0
IDADEMAE         0
ESTCIVMAE      315
ESCMAE         310
QTDFILVIVO    1566
GESTACAO      1216
GRAVIDEZ        76
CONSULTAS        0
APGAR5           0
dtype: int64

In [6]:
# 5) seu código aqui
df["ESTCIVMAE"] = df["ESTCIVMAE"].fillna(9)
df["CONSULTAS"] = df["CONSULTAS"].fillna(9)
df.isna().sum()

LOCNASC          0
IDADEMAE         0
ESTCIVMAE        0
ESCMAE         310
QTDFILVIVO    1566
GESTACAO      1216
GRAVIDEZ        76
CONSULTAS        0
APGAR5           0
dtype: int64

In [7]:
# 6) Seu código aqui
df["QTDFILVIVO"] = df["QTDFILVIVO"].fillna(0)
df.isna().sum()

LOCNASC          0
IDADEMAE         0
ESTCIVMAE        0
ESCMAE         310
QTDFILVIVO       0
GESTACAO      1216
GRAVIDEZ        76
CONSULTAS        0
APGAR5           0
dtype: int64

In [8]:
# 7) seu código aqui
#Poderia usar o dropna nessa variavel que nao interferiria por ser um valor infimo mas para uma pessoa dar entrada nesse arquivo de dados a pessoa precisa esta gravida entao considerou-se o nan como "Única".
df["GRAVIDEZ"].value_counts()



Única              26356
Dupla                488
Tríplice e mais        5
Name: GRAVIDEZ, dtype: int64

In [9]:
df["GRAVIDEZ"] = df["GRAVIDEZ"].fillna('Única')
df["GRAVIDEZ"].isna().sum()

0

In [10]:
#Relacionou o numero de CONSULTAS das pacientes com a semana de GESTACAO para criar uma condicao de ffill e os nan restantes usou dropna.
df["GESTACAO"].value_counts()

37 a 41 semanas        22523
32 a 36 semanas         2234
42 semanas e mais        698
28 a 31 semanas          162
22 a 27 semanas           85
Menos de 22 semanas        7
Name: GESTACAO, dtype: int64

In [11]:
dados_ausentes = df.loc[df['GESTACAO'].isna()]
print(dados_ausentes)

       LOCNASC  IDADEMAE  ESTCIVMAE       ESCMAE  QTDFILVIVO GESTACAO  \
1695         1        27        1.0  8 a 11 anos         1.0      NaN   
2201         1        26        1.0  8 a 11 anos         1.0      NaN   
2359         1        28        2.0   4 a 7 anos         2.0      NaN   
2361         1        33        1.0   4 a 7 anos         4.0      NaN   
2362         1        15        1.0   4 a 7 anos         1.0      NaN   
...        ...       ...        ...          ...         ...      ...   
26782        1        19        5.0  8 a 11 anos         1.0      NaN   
26786        3        24        5.0   4 a 7 anos         1.0      NaN   
26788        1        21        5.0   4 a 7 anos         0.0      NaN   
26800        1        24        5.0  8 a 11 anos         1.0      NaN   
26855        1        25        5.0  8 a 11 anos         2.0      NaN   

      GRAVIDEZ  CONSULTAS  APGAR5  
1695     Única          3     9.0  
2201     Única          1     8.0  
2359     Única 

In [12]:
#Numero de CONSULTAS entre 1 e 4
df.loc[df['CONSULTAS'] == 1, 'GESTACAO'].value_counts()


37 a 41 semanas      547
32 a 36 semanas      114
28 a 31 semanas       13
22 a 27 semanas       13
42 semanas e mais     10
Name: GESTACAO, dtype: int64

In [13]:
df.loc[df['CONSULTAS'] == 4, 'GESTACAO'] = df.loc[df['CONSULTAS'] == 4, 'GESTACAO'].fillna("37 a 41 semanas").ffill()
df.loc[df['CONSULTAS'] == 3, 'GESTACAO'] = df.loc[df['CONSULTAS'] == 3, 'GESTACAO'].fillna("32 a 36 semanas").ffill()
df.loc[df['CONSULTAS'] == 2, 'GESTACAO'] = df.loc[df['CONSULTAS'] == 2, 'GESTACAO'].fillna("28 a 31 semanas").ffill()
df.loc[df['CONSULTAS'] == 1, 'GESTACAO'] = df.loc[df['CONSULTAS'] == 1, 'GESTACAO'].fillna("22 a 27 semanas").ffill()
df.loc[df['CONSULTAS'] == 1, 'GESTACAO'] = df.loc[df['CONSULTAS'] == 1, 'GESTACAO'].fillna("Menos de 22 semanas").ffill()
df["GESTACAO"].isna().sum()

9

In [14]:
df = df.dropna(subset=["GESTACAO"])
df["GESTACAO"].isna().sum()

0

In [15]:
#Por ser um valor infimo usou o dropna
df["ESCMAE"].value_counts()


8 a 11 anos        15578
12 anos ou mais     5754
4 a 7 anos          4725
1 a 3 anos           509
Nenhuma               40
Name: ESCMAE, dtype: int64

In [16]:
df["ESCMAE"].isna().sum()

310

In [17]:
df = df.dropna(subset=["ESCMAE"])
df["ESCMAE"].isna().sum()

0

In [18]:
#8) seu código aqui
df.loc[df['APGAR5'] <= 3, 'classificacao'] = 'asfixia severa'
df.loc[(df['APGAR5'] >= 4) & (df['APGAR5'] <= 5),
           'classificacao'] = 'asfixia moderada'
df.loc[(df['APGAR5'] >= 6) & (df['APGAR5'] <= 7),
           'classificacao'] = 'asfixia leve'
df.loc[(df['APGAR5'] >= 8), 'classificacao'] = 'normal'

df['classificacao'].value_counts()

<ipython-input-18-6ab21adc0916>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[df['APGAR5'] <= 3, 'classificacao'] = 'asfixia severa'


normal              26150
asfixia leve          317
asfixia severa         74
asfixia moderada       65
Name: classificacao, dtype: int64

In [19]:
#9) seu código aqui
df = df.rename(columns=lambda x: x.lower().replace(' ', '_'))
df

,locnasc,idademae,estcivmae,escmae,qtdfilvivo,gestacao,gravidez,consultas,apgar5,classificacao
0,1,19,5.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,10.0,normal
1,1,29,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,9.0,normal
2,1,37,9.0,8 a 11 anos,2.0,37 a 41 semanas,Única,4,10.0,normal
3,1,30,5.0,12 anos ou mais,0.0,37 a 41 semanas,Única,3,10.0,normal
4,1,30,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,10.0,normal
...,...,...,...,...,...,...,...,...,...,...
27023,1,32,2.0,12 anos ou mais,1.0,32 a 36 semanas,Única,4,9.0,normal
27024,1,19,1.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,9.0,normal
27025,1,24,2.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,10.0,normal
27026,1,21,2.0,8 a 11 anos,1.0,32 a 36 semanas,Única,4,9.0,normal
